In [31]:
# Compute Blue Score.
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.translate.meteor_score import meteor_score
from src.train import normalize_answer
from datasets import load_dataset
import pandas as pd

from src.train import *
from ast import literal_eval
import datasets
from sentence_transformers import SentenceTransformer, util
import numpy as np
import nlp
import random
import numpy
import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /home/snajafi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/snajafi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [42]:
#df = pd.read_csv("./t5_all/t5-epoch-on-all-squad.dev.predictions.txt").astype(str)
#df = pd.read_csv("./t5_all/t5-epoch-on-all-squad.dev.predictions.txt").astype(str)
#df = pd.read_csv("./t5_squad/t5-base-predictions.txt").astype(str)
#df = pd.read_csv("./t5_squad/t5-base-predictions.txt").astype(str)
predictions = df["predictions_str"].tolist()
normal_preds = [normalize_answer(pred) for pred in predictions]
dev_refs = read_squad_refs("./squad/dev-v2.0.json")
normal_refs = []
for row in dev_refs:
    temp = []
    for ref in row:
        temp.append(normalize_answer(ref))
    normal_refs.append(temp)

evaluate(normal_refs, normal_preds)

# Only on answerable parts of the squad 2.0.

{'exact_match': 80.02699055330635, 'f1': 88.68562922717916}

In [4]:
from src.pycocoevalcap.meteor.meteor import Meteor
from src.pycocoevalcap.rouge.rouge import Rouge
from src.pycocoevalcap.bleu.bleu import Bleu

meteor_obj = Meteor()
rouge_obj = Rouge()
bleu_obj = Bleu(4)

def nar_test_eval(file_name):
    df = pd.read_csv(file_name).astype(str)
    predictions = df["predictions_str"].tolist()
    normal_preds = [normalize_answer(pred).strip(".") for pred in predictions]

    def process_all_rows(row):
        """Helper function."""
        all_answers = set()
        for asw in row["answers"]:
            answer = asw["text"]
            all_answers.add(normalize_answer(answer).strip("."))
        return {"all_answers": list(all_answers)}

    ref_dev_dataset = load_dataset("narrativeqa", split="test")
    ref_dev_dataset = ref_dev_dataset.map(process_all_rows, remove_columns=["document", "answers", "question"])
    refs = [row["all_answers"] for row in ref_dev_dataset]

    word_target_dict = {}
    word_response_dict = {}

    for i in range(len(refs)):
        if len(refs[i]) == 2:
            word_target_dict[i] = [refs[i][0], refs[i][1]]
        else:
            word_target_dict[i] = [refs[i][0], refs[i][0]]
        word_response_dict[i] = [normal_preds[i]]

    bleu_score, bleu_scores = bleu_obj.compute_score(
            word_target_dict, word_response_dict)

    bleu1_score, _, _, bleu4_score = bleu_score

    bleu1_scores, _, _, bleu4_scores = bleu_scores

    #meteor_score, meteor_scores = meteor_obj.compute_score(
    #        word_target_dict, word_response_dict)

    rouge_score, rouge_scores = rouge_obj.compute_score(
            word_target_dict, word_response_dict) 

    #print("ROUGE-L", rouge_score)
    #print("BLEU-1", bleu1_score)
    #print("BLEU-4", bleu4_score)
    #print("METEOR: ", meteor_score)
    return {"ROUGE-L": rouge_score, "BLEU-1": bleu1_score, "BLEU-4": bleu4_score}

In [6]:
# T5-base from google fine-tuned on NQ, RACE and Squad!
nar_test_eval("./t5-all-nq.test.csv")

Using custom data configuration default
Reusing dataset narrative_qa (/home/snajafi/.cache/huggingface/datasets/narrative_qa/default/0.0.0/5571769afc7baf3132426991d9d84147d5b60c98320ab0b696e8c9ad2d712646)
Loading cached processed dataset at /home/snajafi/.cache/huggingface/datasets/narrative_qa/default/0.0.0/5571769afc7baf3132426991d9d84147d5b60c98320ab0b696e8c9ad2d712646/cache-0199433687b12080.arrow


{'testlen': 34290, 'reflen': 35134, 'guess': [34290, 23733, 16156, 11015], 'correct': [17671, 7661, 3260, 1441]}
ratio: 0.9759776854328863


{'ROUGE-L': 0.5317532522967355,
 'BLEU-1': 0.5028102188346371,
 'BLEU-4': 0.2511642820439916}

In [12]:
nar_test_eval("./t5-all-nq.test.csv")

# Redo by lowercasing the input data!

NameError: name 'nar_test_eval' is not defined

In [20]:
import copy
import rouge

rouge_l_evaluator = rouge.Rouge(
    metrics=["rouge-l"],
    max_n=4,
    limit_length=True,
    length_limit=128,
    length_limit_type="words",
    apply_avg=True,
    apply_best=True,
    alpha=0.5,
    weight_factor=1.2,
    stemming=True,
)

def rouge_l(p, g):
    return rouge_l_evaluator.get_scores(p, g)

def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, [ground_truth])
        scores_for_ground_truths.append(score)
    if isinstance(score, dict) and "rouge-l" in score:
        max_score = copy.deepcopy(score)
        max_score["rouge-l"]["f"] = round(
            max([score["rouge-l"]["f"] for score in scores_for_ground_truths]), 2
        )
        max_score["rouge-l"]["p"] = round(
            max([score["rouge-l"]["p"] for score in scores_for_ground_truths]), 2
        )
        max_score["rouge-l"]["r"] = round(
            max([score["rouge-l"]["r"] for score in scores_for_ground_truths]), 2
        )
        return max_score
    else:
        return round(max(scores_for_ground_truths), 2)

df = pd.read_csv("./t5-all-nq.test.csv").astype(str)
predictions = df["predictions_str"].tolist()
normal_preds = [normalize_answer(pred).strip(".") for pred in predictions]

def process_all_rows(row):
    """Helper function."""
    all_answers = set()
    for asw in row["answers"]:
        answer = asw["text"]
        all_answers.add(normalize_answer(answer).strip("."))
    return {"all_answers": list(all_answers)}

ref_dev_dataset = load_dataset("narrativeqa", split="test")
ref_dev_dataset = ref_dev_dataset.map(process_all_rows, remove_columns=["document", "answers", "question"])
refs = [row["all_answers"] for row in ref_dev_dataset]

scores = []
for i in range(len(refs)):
    golds_subset = refs[i]
    rouge_l_score = metric_max_over_ground_truths(rouge_l, normal_preds[i], golds_subset)
    scores.append(rouge_l_score["rouge-l"]["f"])

print(100.0 * sum(scores) / len(scores))

Using custom data configuration default
Reusing dataset narrative_qa (/home/snajafi/.cache/huggingface/datasets/narrative_qa/default/0.0.0/5571769afc7baf3132426991d9d84147d5b60c98320ab0b696e8c9ad2d712646)
Loading cached processed dataset at /home/snajafi/.cache/huggingface/datasets/narrative_qa/default/0.0.0/5571769afc7baf3132426991d9d84147d5b60c98320ab0b696e8c9ad2d712646/cache-752c0030374476b8.arrow


56.387799564270395


In [58]:
import copy
import rouge
from ast import literal_eval

rouge_l_evaluator = rouge.Rouge(
    metrics=["rouge-l"],
    max_n=4,
    limit_length=True,
    length_limit=128,
    length_limit_type="words",
    apply_avg=True,
    apply_best=True,
    alpha=0.5,
    weight_factor=1.2,
    stemming=True,
)

def rouge_l(p, g):
    return rouge_l_evaluator.get_scores(p, g)

def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, [ground_truth])
        scores_for_ground_truths.append(score)
    if isinstance(score, dict) and "rouge-l" in score:
        max_score = copy.deepcopy(score)
        max_score["rouge-l"]["f"] = round(
            max([score["rouge-l"]["f"] for score in scores_for_ground_truths]), 2
        )
        max_score["rouge-l"]["p"] = round(
            max([score["rouge-l"]["p"] for score in scores_for_ground_truths]), 2
        )
        max_score["rouge-l"]["r"] = round(
            max([score["rouge-l"]["r"] for score in scores_for_ground_truths]), 2
        )
        return max_score
    else:
        return round(max(scores_for_ground_truths), 2)

df = pd.read_csv("./test.0.processed.predicted.csv").astype(str)
predictions = df["predictions_str"].tolist()
normal_preds = [normalize_answer(pred).strip(".") for pred in predictions]

ref_df = pd.read_csv("./test.0.processed.csv").astype(str)
refs = ref_df["correctAnswers"].tolist()
normal_refs = [[normalize_answer(answer).strip(".") for answer in literal_eval(ref)] for ref in refs]


correct_indices = []
wrong_indices = []
empty_indices = []

scores = []
for i in range(len(normal_refs)):
    golds_subset = normal_refs[i]
    if not golds_subset:
        empty_indices.append(i)
        continue
    elif normal_preds[i] in golds_subset:
        correct_indices.append(i)
    elif normal_preds[i] not in golds_subset:
        wrong_indices.append(i)

    rouge_l_score = metric_max_over_ground_truths(rouge_l, normal_preds[i], golds_subset)
    scores.append(rouge_l_score["rouge-l"]["f"])

print(100.0 * sum(scores) / len(scores))

answerable_refs = []
answerable_preds = []
un_answerable_preds = []
for i in range(len(normal_refs)):
    golds_subset = normal_refs[i]
    if golds_subset:
        answerable_refs.append(golds_subset)
        answerable_preds.append(normal_preds[i])
    else:
        un_answerable_preds.append(normal_preds[i])

evaluate(answerable_refs, answerable_preds)


in_complete_data = {"ref": [], "pred": []}
complete_data = {"ref": [], "pred": []}

for i in range(len(answerable_refs)):
    ans_ref = answerable_refs[i]
    ans_pred = answerable_preds[i]
    if ans_pred not in ans_ref:
        in_complete_data["ref"].append(ans_ref)
        in_complete_data["pred"].append(ans_pred)
    else:
        complete_data["ref"].append(ans_ref)
        complete_data["pred"].append(ans_pred)

print(in_complete_data["ref"][10])
print(in_complete_data["pred"][10])

print(len(complete_data["ref"]))
print(len(un_answerable_preds))


print(len(correct_indices))
print(len(wrong_indices))
print(len(empty_indices))


correct_samples = random.sample(correct_indices, 10)
correct_rows_sample = ref_df.iloc[correct_samples,]
correct_rows_sample_predictions = df.iloc[correct_samples,]

wrong_samples = random.sample(wrong_indices, 10)
wrong_rows_sample = ref_df.iloc[wrong_samples,]
wrong_rows_sample_predictions = df.iloc[wrong_samples,]

empty_samples = random.sample(empty_indices, 10)
empty_rows_sample = ref_df.iloc[empty_samples,]
empty_rows_sample_predictions = df.iloc[empty_samples,]

correct_rows_sample.to_csv("./correct_rows_sample.csv")
correct_rows_sample_predictions.to_csv("./correct_rows_sample.predictions.csv")


wrong_rows_sample.to_csv("./wrong_rows_sample.csv")
wrong_rows_sample_predictions.to_csv("./wrong_rows_sample.predictions.csv")

empty_rows_sample.to_csv("./empty_rows_sample.csv")
empty_rows_sample_predictions.to_csv("./empty_rows_sample.predictions.csv")
#print(wrong_rows_sample)
#print(empty_rows_sample)

#print(random.sample(wrong_indices, 10))
#print(random.sample(empty_indices, 10))

85.14349999999955
['caribbean']
cobra and san pero
4268
6000
4268
1732
6000


In [45]:
#df = pd.read_csv("./t5_on_dream/t5-base-with-nq-dream-inferentials-predictions.txt").astype(str)

df = pd.read_csv("./unifiedqa-inferentials.csv").astype(str)
predictions = df["predictions_str"].tolist()
normal_preds = [normalize_answer(pred) for pred in predictions]
val_contexts, val_answers = read_dream_data('./dream-data-v2/summary-passages-questions-cleaned.inferentials.csv')
refs = [normalize_answer(literal_eval(answer)[0]) for answer in val_answers]

rouge = PerlRouge(rouge_l=True)

scores = rouge.evaluate(normal_preds, refs)

model = SentenceTransformer('stsb-roberta-large')

embedding1 = model.encode(refs, convert_to_tensor=True)
embedding2 = model.encode(normal_preds, convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
sim = 0.0
for i in range(len(refs)):
    sim += cosine_scores[i][i].item()

mean_sim = sim / len(refs)

print(mean_sim)

print(scores)



#df = pd.read_csv("./t5_on_dream/t5-base-with-nq-dream-literal-predictions.txt").astype(str)
df = pd.read_csv("./unifiedqa-literals.csv").astype(str)
predictions = df["predictions_str"].tolist()
normal_preds = [normalize_answer(pred) for pred in predictions]
val_contexts, val_answers = read_dream_data('./dream-data-v2/summary-passages-questions-cleaned.literals.csv')
refs = [normalize_answer(literal_eval(answer)[0]) for answer in val_answers]

rouge = PerlRouge(rouge_l=True)

scores = rouge.evaluate(normal_preds, refs)

print(scores)


model = SentenceTransformer('stsb-roberta-large')

embedding1 = model.encode(refs, convert_to_tensor=True)
embedding2 = model.encode(normal_preds, convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
sim = 0.0
for i in range(len(refs)):
    sim += cosine_scores[i][i].item()

mean_sim = sim / len(refs)

print(mean_sim)

0.34203120798658615
{'rouge-1': {'r': 0.0043, 'r_conf_int': (0.00038, 0.00947), 'p': 0.00038, 'p_conf_int': (2e-05, 0.00082), 'f': 0.00069, 'f_conf_int': (5e-05, 0.00148)}, 'rouge-2': {'r': 0.0, 'r_conf_int': (0.0, 0.0), 'p': 0.0, 'p_conf_int': (0.0, 0.0), 'f': 0.0, 'f_conf_int': (0.0, 0.0)}, 'rouge-l': {'r': 0.0043, 'r_conf_int': (0.00038, 0.00947), 'p': 0.00038, 'p_conf_int': (2e-05, 0.00082), 'f': 0.00069, 'f_conf_int': (5e-05, 0.00148)}}
{'rouge-1': {'r': 0.00507, 'r_conf_int': (0.00299, 0.0075), 'p': 0.00047, 'p_conf_int': (0.00027, 0.0007), 'f': 0.00085, 'f_conf_int': (0.00049, 0.00127)}, 'rouge-2': {'r': 0.0, 'r_conf_int': (0.0, 0.0), 'p': 0.0, 'p_conf_int': (0.0, 0.0), 'f': 0.0, 'f_conf_int': (0.0, 0.0)}, 'rouge-l': {'r': 0.00507, 'r_conf_int': (0.00299, 0.0075), 'p': 0.00047, 'p_conf_int': (0.00027, 0.0007), 'f': 0.00085, 'f_conf_int': (0.00049, 0.00127)}}
0.39351543257466154
